## The following file is used to present the interactive plot with the help of scipy. 
The following code is better than the initial interactive plot but it still cannot be uploaded on github as this file is very heavy. In order to solve this problem we use a Dynamic map in the final interactive plot. 

The following code requires us to define a class in order to call the method again and again in order to make the widget. This can sometimes be a little complicated to understand when going over it for the first time. 

Order of correctness - 

Initial interactive --> Scipy interactive(this file) --> Final interactive plot

In [ ]:
# importing all the packages
import matplotlib.pyplot as plt

from scipy.io import wavfile
import io
import holoviews as hv
import param
from holoviews import opts
import xarray as xr

import os 
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

import imp 
from holoviews import opts
from holoviews.operation import Operation

plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
fs, data = wavfile.read('soundfile.wav')

In [ ]:
data_transpose = data.T

In [ ]:
from scipy import signal
freqs, bins, Pxx = signal.spectrogram(data_transpose[0], nperseg = 20, fs=fs, noverlap=10)

In [ ]:
# this is the block o f code to present just one spectrogram made
plt.pcolormesh(bins, freqs, Pxx)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [ ]:
xr_spec = xr.DataArray(np.log(Pxx),dims = ('freq','time'),coords = {'freq':freqs,'time':bins})
xr_spec.name = 'Spectrogram'
hv_spec = hv.Dataset(xr_spec)

hv.extension('bokeh')
spec = hv_spec.to(hv.Image, ['time', 'freq'])

In [ ]:
# Making a class of the above two functions to call when the holoviews is called 
data_transpose = data.T 

class interactive_plot(Operation):
    
    NFFT = param.Number(default=512)  # the length of the windowing segments
    
    def _process(self, element,  key=None):
        data_transpose = data.T    
        plt.figure(figsize = (20,10))
        #fig, ax1 = plt.subplots(nrows=1)
        freqs, bins, Pxx = signal.spectrogram(data_transpose[0], nperseg = self.p.NFFT, fs=fs, noverlap=10)
        
        xr_spec = xr.DataArray(np.log(Pxx),dims = ('freq','time'),coords = {'freq':freqs,'time':bins})
        xr_spec.name = 'Spectrogram'
        dataset = hv.Dataset(xr_spec)
        spectrogram_image = dataset.to(hv.Image, ['time', 'freq'])
        spectrogram_image
        
        return spectrogram_image

In [ ]:
hv.extension('bokeh', 'matplotlib')

hv.extension('bokeh')
output_notebook()

imp.reload(hv)

widget = hv.HoloMap({NFFT: interactive_plot(spec, NFFT=NFFT) 
                      for NFFT in range(16, 512, 8)}, kdims="NFFT")

widget.opts(framewise=True)